# Determine the fraction of fuel-to-cargo transfers for ATB/ITBs

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [5]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

oil_attribution_dir  = '/Users/rmueller/Data/MIDOSS/marine_transport_data/'
oil_attribution_file = 'oil_attribution.yaml'

with open(f'{oil_attribution_dir}{oil_attribution_file}') as file:
    oil_attribution = yaml.safe_load(file)

vessel_types = oil_attribution['categories']['tank_vessels']
fuel_types   = oil_attribution['categories']['fuel_type']
facilities   = oil_attribution['categories']['US_origin_destination']

In [6]:
# Import columns are: (G) Deliverer, (H) Receiver, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")

## Estimate the fuel to cargo transfers for ATBs and the fuel to cargo transfers for non-ATB tank barges

In [10]:
df.head(1)

,Deliverer,Receiver,Product,TransferQtyInGallon,TransferType,DelivererTypeDescription,ReceiverTypeDescription
0,Maxum (Rainer Petroleum),VICTORIA CLIPPER IV,DIESEL/MARINE GAS OIL,2500,Fueling,Facility,PASSENGER SHIP


In [12]:
transfer_types = df.loc[
    (df.Receiver.isin(facilities) | 
     df.Deliverer.isin(facilities)),
    ['TransferType','TransferQtyInGallon']
].groupby('TransferType').count()
transfer_types

,TransferQtyInGallon
TransferType,
Cargo,4434
Fueling,2780
Lightering,3
Other (incl Lube/Hyd Oil),407
Vacuum,21


In [15]:
transfer_types = df[{'TransferType','TransferQtyInGallon'}].groupby('TransferType').count()
transfer_types

,TransferQtyInGallon
TransferType,
Cargo,4632
Fueling,8769
Lightering,18
Other (incl Lube/Hyd Oil),982
Vacuum,309


In [28]:
deliverer_types = df[{'DelivererTypeDescription','TransferQtyInGallon'}].groupby('DelivererTypeDescription').count()
deliverer_types

,TransferQtyInGallon
DelivererTypeDescription,
BULK CARRIER,3
CONTAINER SHIP,5
FERRY VESSEL,2
FISH PROCESSOR,2
FISHING VESSEL,8
FISHING VESSEL - ATA,2
Facility,7806
Mobile,3032
NOT COVERED VESSEL,34


#### Take-away: A lot of fueling for washington vessels occurs away from marine terminals (presumably fueling barges) for oil transfers > 1000 liters.  61.5% of marine terminal transfers are cargo and 38.5% are fueling compared to 34.6% of all transfers being cargo and 65.4% of all transfers being fuel

In [30]:
DOE_transfers = {}
DOE_transfers['barge'] = {}
DOE_transfers['atb'] = {}

barge_facility = df.loc[
    (df.Receiver.isin(facilities) | 
     df.Deliverer.isin(facilities)) &
    (df.DelivererTypeDescription.isin(['TANK BARGE','TUGBOAT']) |
     df.ReceiverTypeDescription.isin(['TANK BARGE','TUGBOAT'])) &
    (~df.Deliverer.str.contains('ITB')) & 
    (~df.Deliverer.str.contains('ATB')) &
    (~df.Receiver.str.contains('ITB')) & 
    (~df.Receiver.str.contains('ATB')),
    ['TransferType','TransferQtyInGallon']
].groupby('TransferType').count()     
DOE_transfers['barge']['facility_tally'] = barge_facility['TransferQtyInGallon']

barge_fromvessel = df.loc[
    (df.DelivererTypeDescription.isin(['Mobile', 'TANK BARGE', 'TANK SHIP'])) &
    (df.ReceiverTypeDescription.isin(['TANK BARGE','TUGBOAT'])) &
    (~df.Receiver.str.contains('ITB')) & 
    (~df.Receiver.str.contains('ATB')),
    ['TransferType','TransferQtyInGallon']
].groupby('TransferType').count()     
DOE_transfers['barge']['vessel_tally'] = barge_fromvessel['TransferQtyInGallon']

atb_facility = df.loc[
    (df.Receiver.isin(facilities) | 
     df.Deliverer.isin(facilities)) &
    (df.DelivererTypeDescription.isin(['TANK BARGE','TUGBOAT']) |
     df.ReceiverTypeDescription.isin(['TANK BARGE','TUGBOAT'])) &
    (df.Deliverer.str.contains('ITB') | df.Deliverer.str.contains('ATB') |
     df.Receiver.str.contains('ITB') | df.Receiver.str.contains('ATB')),
    ['TransferType','TransferQtyInGallon']
].groupby('TransferType').count()
DOE_transfers['atb']['facility_tally'] = atb_facility['TransferQtyInGallon']

atb_fromvessel = df.loc[
    (df.DelivererTypeDescription.isin(['Mobile', 'TANK BARGE', 'TANK SHIP'])) &
    (df.Receiver.str.contains('ITB') | df.Receiver.str.contains('ATB')),
    ['TransferType','TransferQtyInGallon']
].groupby('TransferType').count()     
DOE_transfers['atb']['vessel_tally'] = atb_fromvessel['TransferQtyInGallon']

print('-------- Number of barge transfers to/from facilities--------')
print(DOE_transfers['barge']['facility_tally'])

print('-------- Number of ATB transfers to/from facilities --------')
print(DOE_transfers['atb']['facility_tally'])

print('-------- Number of barge transfers from other vessels--------')
print(DOE_transfers['barge']['vessel_tally'])

print('-------- Number of ATB transfers from other vessels --------')
print(DOE_transfers['atb']['vessel_tally'])

-------- Number of barge transfers to/from facilities--------
TransferType
Cargo                        2770
Fueling                       206
Lightering                      2
Other (incl Lube/Hyd Oil)      45
Vacuum                         16
Name: TransferQtyInGallon, dtype: int64
-------- Number of ATB transfers to/from facilities --------
TransferType
Cargo                        675
Fueling                        3
Other (incl Lube/Hyd Oil)      9
Name: TransferQtyInGallon, dtype: int64
-------- Number of barge transfers from other vessels--------
TransferType
Cargo                        32
Fueling                      11
Lightering                   10
Other (incl Lube/Hyd Oil)     4
Name: TransferQtyInGallon, dtype: int64
-------- Number of ATB transfers from other vessels --------
TransferType
Cargo                         1
Fueling                      48
Lightering                    1
Other (incl Lube/Hyd Oil)     6
Name: TransferQtyInGallon, dtype: int64


In [23]:
 DOE_in['atb']['TransferQtyInGallon']

TransferType
Cargo                        207
Other (incl Lube/Hyd Oil)      9
Name: TransferQtyInGallon, dtype: int64

In [24]:
 DOE_out['atb']['TransferQtyInGallon']

TransferType
Cargo      468
Fueling      3
Name: TransferQtyInGallon, dtype: int64

##  Below is my original analysis, which I'm keeping here as reference

## Extract **cargo** data transferred from vessel to facility

In [4]:
# list of names in DOE dataset that are used as "DelivererTypeDescription" for vessel transfers
vessel_names = ['TANK BARGE', 'TUGBOAT']
# Get all cargo fuel transfers
bool_cargo = df['TransferType']=='Cargo'
cargo_data = df[bool_cargo]
cargo_from_facility = cargo_data[ cargo_data['DelivererTypeDescription']=='Facility' ]
cargo_to_barge  = cargo_from_facility[ cargo_from_facility['ReceiverTypeDescription'] == 'TANK BARGE' ] 
cargo_to_tug    = cargo_from_facility[ cargo_from_facility['ReceiverTypeDescription'] == 'TUGBOAT' ]

cargo_barge = pd.concat([cargo_to_barge, cargo_to_tug])

# reset indices so they increase chronologically from 0 instead of keeping parent index values
cargo_from_facility.reset_index( drop=True, inplace=True )
cargo_barge.reset_index( drop=True, inplace=True )
cargo_data.reset_index( drop=True, inplace=True )

## Extract **fuel** data transferred from vessel to facility

In [5]:
# list of names in DOE dataset that are used as "DelivererTypeDescription" for vessel transfers
vessel_names = ['TANK BARGE', 'TUGBOAT']
# Get all cargo fuel transfers
bool_fuel = df['TransferType']=='Fueling'
fuel_data = df[bool_fuel]
fuel_from_facility = fuel_data[ fuel_data['DelivererTypeDescription']=='Facility' ]
fuel_to_barge  = fuel_from_facility[ fuel_from_facility['ReceiverTypeDescription'] == 'TANK BARGE' ] 
fuel_to_tug    = fuel_from_facility[ fuel_from_facility['ReceiverTypeDescription'] == 'TUGBOAT' ]

fuel_barge = pd.concat([fuel_to_barge, fuel_to_tug])

# reset indices so they increase chronologically from 0 instead of keeping parent index values
fuel_from_facility.reset_index( drop=True, inplace=True )
fuel_barge.reset_index( drop=True, inplace=True )
fuel_data.reset_index( drop=True, inplace=True )

In [6]:
# just to check that a vessel classified as "tank barge" may receive fuel 
fuel_to_barge.shape

(31, 7)

In [7]:
fuel_to_tug.shape

(184, 7)

In [8]:
cargo_to_barge.shape

(2451, 7)

In [9]:
cargo_to_tug.shape

(8, 7)

In [10]:
print(f'Total amount of oil used to fuel vessels from WA facilities: {fuel_from_facility.TransferQtyInGallon.sum():4.2e} gallons')

Total amount of oil used to fuel vessels from WA facilities: 6.49e+07 gallons


In [11]:
print(f'Total amount of oil transferred as cargo from WA facilities: {cargo_from_facility.TransferQtyInGallon.sum():4.2e} gallons')

Total amount of oil transferred as cargo from WA facilities: 4.42e+09 gallons


In [12]:
print(f'Percent of fuel/cargo from WA facilities: {100*fuel_from_facility.TransferQtyInGallon.sum()/cargo_from_facility.TransferQtyInGallon.sum():4.2f} %')

Percent of fuel/cargo from WA facilities: 1.47 %


### Let's make sure that the numbers are similar for fueling if we remove the 'facilities' requirement
Fueling can happen at locations not deemed "facility".  These include fuel docks or re-distribution vessels.  This number for cargo will likely double count cargo as going from land to redistribution vessel and then from redistribution vessel to another vessel, so I don't evaluate that number here.

In [13]:
print(f'Total amount of oil used to fuel vessels in WA waters: {fuel_data.TransferQtyInGallon.sum():4.2e} gallons')

Total amount of oil used to fuel vessels in WA waters: 6.80e+08 gallons


In [14]:
print(f'Percent of fueling/cargo-transport in WA waters: {100*fuel_data.TransferQtyInGallon.sum()/cargo_from_facility.TransferQtyInGallon.sum():4.2f} %')

Percent of fueling/cargo-transport in WA waters: 15.38 %


In [15]:
print('**  Wow.  That is a big difference.  Let me highlight it here  **\n' \
f'Percent of fuel/cargo from WA facilities: {100*fuel_from_facility.TransferQtyInGallon.sum()/cargo_from_facility.TransferQtyInGallon.sum():4.2f} %\n'\
f'Percent of fueling/cargo-transport in WA waters: {100*fuel_data.TransferQtyInGallon.sum()/cargo_from_facility.TransferQtyInGallon.sum():4.2f} %')

**  Wow.  That is a big difference.  Let me highlight it here  **
Percent of fuel/cargo from WA facilities: 1.47 %
Percent of fueling/cargo-transport in WA waters: 15.38 %


### Noted: Lots of fueling at non-oil-transfer facilities.  I'm going to start this analysis based on Bunkering at facilities; but, in lieu of the above information, I will need to verify that oil tugs bunker at oil-transfer facilities. The above difference may capture the significance of ferry and recreational traffic.

## Evaluate ATB bunkering and cargo transfers from WA facilities
- start with "from facility" sub-samples (fuel_from_facilities, cargo_from_facilities)
- determine bunkering transfers from facilities
- determine cargo transfers from facilities

In [16]:
# loop through entire record of fuel transfers from facilities
[nrows,ncols] = fuel_from_facility.shape

# create ATB dataframe for fuel transfers from facilities
fuel_atb = pd.DataFrame(columns = fuel_from_facility.columns)

for row in range(nrows):
    if 'ITB' in fuel_from_facility.Receiver[row] or 'ATB' in fuel_from_facility.Receiver[row]:
        fuel_atb = fuel_atb.append( fuel_from_facility.iloc[row] )

# reset indexing        
fuel_atb.reset_index( drop=True, inplace=True )

In [17]:
# loop through entire record of cargo transfers from facilities
[nrows,ncols] = cargo_from_facility.shape

# create ATB dataframe for fuel transfers from facilities
cargo_atb = pd.DataFrame(columns = cargo_from_facility.columns)

for row in range(nrows):
    if 'ITB' in cargo_from_facility.Receiver[row] or 'ATB' in cargo_from_facility.Receiver[row]:
        cargo_atb = cargo_atb.append( cargo_from_facility.iloc[row] )

# reset indexing        
cargo_atb.reset_index( drop=True, inplace=True )

In [18]:
len(fuel_atb)

16

In [19]:
len(cargo_atb)

470

In [20]:
print(f'The ratio of fuel transfers from oil transfer facility to ATB vs. cargo transfers is: {len(fuel_atb)/len(cargo_atb):4.3f}\n'\
     f'Total number of ATB fuel transfers from oil-transfer facilities: {len(fuel_atb)}\n'\
     f'Total number of ATB cargo transfers from oil-transfer facilities: {len(cargo_atb)}')

The ratio of fuel transfers from oil transfer facility to ATB vs. cargo transfers is: 0.034
Total number of ATB fuel transfers from oil-transfer facilities: 16
Total number of ATB cargo transfers from oil-transfer facilities: 470


### Test the hypthesis that oil cargo ATBs only fuel-up at oil-transfer facilities

In [21]:
# create list of ATBs/ITBs receiving cargo from oil transfer facilities
atb_names = {}
atb_names['fuel']  = []
atb_names['cargo'] = []

[rows,cols] = cargo_atb.shape
# expand list of atb operators
for row in range(rows):
    if cargo_atb.Receiver[row] not in atb_names['cargo']:
        atb_names['cargo'].append(cargo_atb.Receiver[row])
        
[rows,cols] = fuel_atb.shape
# expand list of atb operators
for row in range(rows):
    if fuel_atb.Receiver[row] not in atb_names['fuel']:
        atb_names['fuel'].append(fuel_atb.Receiver[row])

In [22]:
atb_names['cargo']

['ATB BARGE KIRBY 185-01',
 'ATB BARGE 550-2',
 'ATB BARGE FIGHT FANCONI ANEMIA',
 'ATB BARGE 650-10',
 'ATB BARGE DBL 185',
 'ATB BARGE ONEDREAM',
 'ATB BARGE 650-7',
 'ATB BARGE 550-3',
 'ATB BARGE ZIDELL MARINE 277',
 'ATB BARGE ALL ABOARD FOR A CURE',
 'ATB BARGE EDWARD ITTA',
 'ITB SUPPLIER',
 'ATB BARGE DR. ROBERT J. BEALL',
 'ITB VANCOUVER',
 'ATB BARGE DBL185',
 'ATB BARGE 650-2',
 'ATB BARGE DBL 78']

In [23]:
atb_names['fuel']

['ATB TUG BILL GOBEL',
 'ATB TUG MIN ZIDELL',
 'ATB TUG JAKE SHEARER',
 'ATB TUG VISION',
 'ATB TUG SOUND RELIANCE',
 'ATB TUG OCEAN RELIANCE']

### create list of all fueling and cargo transfers regardless as to wether from "facility"

In [24]:
# loop through all names of tugs receiving fuel from facilities and see if they get fuel elsewhere too
oilcargo_atb_fueling = pd.DataFrame(columns = cargo_from_facility.columns)
oilcargo_atb_cargo   = pd.DataFrame(columns = cargo_from_facility.columns)

[rows,cols] = fuel_data.shape
for row in range(rows): 
    if fuel_data.Receiver[row] in atb_names['fuel']:
        oilcargo_atb_fueling = oilcargo_atb_fueling.append( fuel_data.iloc[row] )
        
[rows,cols] = cargo_data.shape
for row in range(rows): 
    if cargo_data.Receiver[row] in atb_names['cargo']:
        oilcargo_atb_cargo = oilcargo_atb_cargo.append( cargo_data.iloc[row] )       

### print names of non-facility transfer sights for fueling and cargo

In [25]:
oilcargo_atb_cargo.reset_index( drop=True, inplace=True )
for row in range(len(oilcargo_atb_cargo)):
    if 'Facility' not in oilcargo_atb_cargo.DelivererTypeDescription[row]:
        print(oilcargo_atb_cargo.Deliverer[row])

ATB BARGE 550-2


In [26]:
oilcargo_atb_fueling.reset_index( drop=True, inplace=True )
for row in range(len(oilcargo_atb_fueling)):
    if 'Facility' not in oilcargo_atb_fueling.DelivererTypeDescription[row]:
        print(oilcargo_atb_fueling.Deliverer[row])

HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
GLOBAL PILOT
DAGWOOD
GLOBAL PILOT
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
GLOBAL PILOT
HMS 26-1
GLOBAL PROVIDER
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1
HMS 26-1


### take away: Lots of ATB fueling by HMS 26-1!

## Add up total number of fuelings to ATB tugs and total number of cargo transfers to ATB tank barges

In [27]:
# query all fueling transfers
total_fueling_bytug={}
total_fueling = 0
for tug in atb_names['fuel']:
    bool_tug = fuel_data['Receiver'] == tug
    total_fueling_bytug[tug] = fuel_data[bool_tug]
    total_fueling += len(total_fueling_bytug[tug])

In [38]:
# query cargo transfers from facilities only
total_cargo_bybarge={}
total_cargo = 0
for barge in atb_names['cargo']:
    bool_barge = cargo_from_facility['Receiver'] == barge
    total_cargo_bybarge[barge] = cargo_from_facility[bool_barge]
    total_cargo += len(total_cargo_bybarge[barge])

In [39]:
print(f'total bunkering count of tugs identified as ATB tugs (includes non-facility fueling): {total_fueling}\n'\
      f'total cargo transfers to barges identified as ATBs (includes 1 non-facility transfer): {total_cargo}\n'
     f'Ratio of ATB cargo/bunkering transfers: {total_cargo/total_fueling:4.2f}')

total bunkering count of tugs identified as ATB tugs (includes non-facility fueling): 44
total cargo transfers to barges identified as ATBs (includes 1 non-facility transfer): 470
Ratio of ATB cargo/bunkering transfers: 10.68


## Let's evaluate more general case of tug and barge transfers

In [30]:
fuel_to_barge  = fuel_data[ fuel_data['ReceiverTypeDescription'] == 'TANK BARGE'] 
fuel_to_barge  = fuel_to_barge.append(fuel_data[ fuel_data['ReceiverTypeDescription'] == 'TUGBOAT'] )
fuel_to_barge.reset_index(drop=True, inplace=True)
print(f'total bunkering count of all tugs: {len(fuel_to_barge)}')

total bunkering count of all tugs: 242


In [32]:
cargo_to_barge  = cargo_data[ cargo_data['ReceiverTypeDescription'] == 'TANK BARGE'] 
cargo_to_barge  = cargo_to_barge.append(cargo_data[ cargo_data['ReceiverTypeDescription'] == 'TUGBOAT'] )
cargo_to_barge.reset_index(drop=True, inplace=True)
print(f'total bunkering count of all tugs: {len(fuel_to_barge)}\n'\
      f'total oil cargo transfers of all tugs: {len(cargo_to_barge)}\n'\
     f'Ratio of ATB cargo/bunkering transfers: {len(cargo_to_barge)/len(fuel_to_barge):4.2f}')

total bunkering count of all tugs: 242
total oil cargo transfers of all tugs: 2495
Ratio of ATB cargo/bunkering transfers: 10.31


In [36]:
## Get a list of all tugs involved in this fueling 
barge_list = []
[nrows,ncols] = fuel_to_barge.shape
for row in range(nrows):
    if fuel_to_barge.Receiver[row] not in barge_list:
        barge_list.append(fuel_to_barge.Receiver[row])

In [37]:
barge_list

['WEBB MOFFETT',
 'ATB BARGE ONEDREAM',
 'EXPLORER',
 'OLYMPIC SPIRIT',
 'ATLAS',
 'GLOBAL PILOT',
 'BARGE FAIRWEATHER',
 'GLENN',
 'SHAUNA KAY',
 'DOTTIE',
 'YON-322',
 'ROGUE',
 'USN SWOB 25',
 'HMS 2000',
 'PB-32',
 'DUGAN PEARSALL',
 'VIJAY SEA',
 'YON-326',
 'COMMENCEMENT BAY',
 'HMS 26-1',
 'BARGE ST. ELIAS',
 'DOUBLE SKIN 311',
 'GRANITE POINT',
 'RYAN POINT',
 'THE CHIEF',
 'ATB TUG VISION',
 'SUNDIAL',
 'HURRICANE',
 'CROWN POINT',
 'CAPTAIN BOB',
 'ATB TUG RESOLVE',
 'GARTH FOSS',
 'ISLAND VIKING',
 'LINDSEY FOSS',
 'PACIFIC PRIDE',
 'ARCTIC TAGLU',
 'TIDEWATER']

In [45]:
[rows,cols] = fuel_data.shape 
for row in range(rows):
    if fuel_data.Receiver[row]=='ATB TUG JAKE SHEARER':
        print(fuel_data.Receiver[row])


ATB TUG JAKE SHEARER
ATB TUG JAKE SHEARER
ATB TUG JAKE SHEARER
ATB TUG JAKE SHEARER
ATB TUG JAKE SHEARER
ATB TUG JAKE SHEARER
